<a href="https://colab.research.google.com/github/yosarawut/notebook/blob/main/predict_multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ทดสอบการ Predict จากข้อมูลใบขนสินค้าจากระบบ
===

In [ ]:
!pip install  time()

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip install  time()'


In [1]:
import pandas as pd
import numpy as np
from time import time
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier


#urls = 'https://github.com/dragon-library/work_space/raw/main/HS_Code/Lab/data/hs_code.xlsx'
urls = 'data/test_hs_code.xlsx'
types = 'section'
#types = "chapter"

def get_master(sheets,types = 'section'):
    data = pd.read_excel(urls,sheet_name= sheets)
    data[types] = data[types].map('{:02}'.format)
    data = data[[types,'description']]
    data['description'] = data['description'].str.lower()
 #   data = data.rename(columns={'heading' : 'target', 'product_desc' : 'question_text'})
     

    return data

def manage_data(df):
    df.columns = ['target', 'data']   
    
    
    return df

In [17]:
%%time
types = "section"
#types = "chapter"

print("Load the dataset: Section")

t0 = time()

sheets = '8_digit'
eights = get_master(sheets,types)
sheets = '6_digit'
sixs = get_master(sheets,types)
sheets = '4_digit'
fours = get_master(sheets,types)
sheets = '2_digit'
twos = get_master(sheets,types)


sheets = 'imp'
imp = get_master(sheets,types)

data = pd.concat([eights,sixs,fours,twos,exp], ignore_index=True)
master = manage_data(data)



load_time = time() - t0
print("Load dataset time:  %0.3fs" % load_time)

print("size : ", len(master))



Load the dataset: Section
Load dataset time:  139.912s
size :  81181
Wall time: 2min 19s


In [18]:
master.head()

,target,data
0,01,"horses; live, purebred breeding animals - pure..."
1,01,"horses; live, other than purebred breeding ani..."
2,01,asses; live - other
3,01,mules and hinnies; live- other
4,01,"cattle; live, purebred breeding animals - pure..."


In [15]:
master.shape[0]

81181

In [16]:
df.iloc[0][1]

'horses; live, purebred breeding animals - purebred breeding animals'

In [19]:
from collections import Counter
Counter(master["target"])

Counter({'01': 987,
         '02': 889,
         '03': 247,
         '04': 855,
         '05': 989,
         '06': 4285,
         '07': 6176,
         '08': 334,
         '09': 447,
         '10': 869,
         '11': 3465,
         '12': 338,
         '13': 702,
         '14': 158,
         '15': 33655,
         '16': 18112,
         '17': 3661,
         '18': 3681,
         '19': 56,
         '20': 1253,
         '21': 22})

In [20]:
# Data Cleaning
def clean_str(text):
    # remove everything except alphabets
    text = re.sub("[^a-zA-Z]", "", text)
    # remove whitespaces
    text = ' '.join(text.split())
    text = text.lower()
    
    return text

import re 
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    
    return string.strip().lower()

In [21]:
print("train test split dataset")
#train test split
df = master.copy()

from sklearn.model_selection import train_test_split
X = []
for i in range(df.shape[0]):
    X.append(clean_str(df.iloc[i][1]))
y = np.array(df["target"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

train test split dataset


In [23]:
%%time
#feature engineering and model selection
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

print("Training: ")

#pipeline of feature engineering and model
t0 = time()
model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

print("paramater selection")
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

Training: 
paramater selection


C:\Users\ASUS\.conda\envs\env_ml\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.45649898962890817
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 1)}
Wall time: 26min


In [26]:
%%time
#preparing the final pipeline using the selected parameters
print("preparing the final pipeline using the selected parameters")
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#fit model with training data
print("fit model with training data")
model.fit(X_train, y_train)
train_time = time() - t0
print("train time: %0.3fs" % train_time)

#evaluation on test data
t0 = time()
pred = model.predict(X_test)
test_time = time() - t0
print("test time:  %0.3fs" % test_time)
print(" Target \n ", model.classes_)

preparing the final pipeline using the selected parameters
fit model with training data


C:\Users\ASUS\.conda\envs\env_ml\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train time: 2689.726s
test time:  0.179s
 Target 
  ['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21']
Wall time: 1min 14s


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[  21,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,   21,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,   14,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,   28,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,   44,    5,    2,    0,    0,    0,    0,
           0,    0,    0,    0,    1,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    3,  193,   10,    0,    0,    0,    0,
           0,    0,    0,    1,    2,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    1,    5,  696,    0,    1,    0,   37,
           0,    1,    0,   47,   51,    0,   12,    0,    0,    0],

In [28]:
## เริ่ีม predict

accuracy_score(y_test, pred)

0.5744610962841306

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          01       1.00      0.07      0.12       320
          02       1.00      0.08      0.14       276
          03       1.00      0.17      0.30        80
          04       1.00      0.12      0.21       243
          05       0.85      0.16      0.26       281
          06       0.92      0.15      0.26      1290
          07       0.82      0.37      0.51      1870
          08       0.88      0.14      0.25        97
          09       1.00      0.16      0.27       147
          10       0.93      0.15      0.26       282
          11       0.92      0.20      0.32       973
          12       0.97      0.32      0.48        93
          13       0.87      0.19      0.31       205
          14       1.00      0.11      0.20        37
          15       0.50      0.97      0.66     10151
          16       0.88      0.30      0.45      5433
          17       0.99      0.70      0.82      1098
          18       0.72    

In [ ]:
#save the model
print("Save Model")
import joblib
#joblib.dump(model, 'model_section.pkl', compress=1)